# **0. Basics**
Importing required modules and Defining some useful functions.

In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.animation as animation
from IPython.display import HTML

# For utilities
import os, shutil, time, sys

# For conversion
sys.path.insert(0, '../../')
import opencv_transforms.transforms as TF
import opencv_transforms.functional as FF
import dataloader
from PIL import Image
import cv2
import numbers

# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as vutils
import random
import tqdm
from torch.autograd import Variable

# For our model
import mymodels
import torchvision.models
from torchvision import datasets, transforms

# To ignore warning
import warnings
warnings.simplefilter("ignore", UserWarning)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if torch.cuda.is_available():
    print("The gpu to be used : {}".format(torch.cuda.get_device_name(0)))
else:
    print("No gpu detected")

The gpu to be used : GeForce GTX 1060 6GB


# **1. Loading dataset**

## 1.1 Dataloader

The code of dataloader to load input videos is written in `dataloader.py`. Loaded data is 5D tensor with size of **[N, C, L, W, H]**.

To pre-process input data, the module `opencv_transforms.transforms` and `opencv_transforms.functional` are imported. These are implemented with **openCV** so faster than `torchvision.transforms` which is based on **Pillow**.[2]

[2] Jim Bohnslav,"opencv_transforms",https://github.com/jbohnslav/opencv_transforms,2020.1.13.

In [2]:
# batch_size
batch_size=4
clip_length=16
sampling_rate=4

# Training
print('Loading Training data...', end=' ')
train_transforms = TF.Compose([
    TF.Resize((121, 178))
    ])
train_imagefolder = dataloader.VideoFolder(
    '../../dataset/aps_cut/train', 
    transform=train_transforms, 
    extensions=('avi'), 
    clip_length=clip_length,
    sampling_rate=sampling_rate,
    start_random=True,
    )
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
train_batch = next(iter(train_loader))
print("Done!")
print("Training data size : {}".format(len(train_imagefolder)))

# Validation
print('Loading Validation data...', end=' ')
val_transforms = TF.Compose([
    TF.Resize((121, 178)),
    ])
val_imagefolder = dataloader.VideoFolder(
    '../../dataset/aps_cut/val', 
    transform=val_transforms, 
    extensions=('avi'), 
    clip_length=clip_length,
    sampling_rate=sampling_rate,
    start_random=False,
    )
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)
val_batch = next(iter(val_loader))
print("Done!")
print("Validation data size : {}".format(len(val_imagefolder)))

Loading Training data... Done!
Training data size : 2543
Loading Validation data... Done!
Validation data size : 679


## 1.2 Dataset Test

Check the dataset.

In [46]:
temp_batch_iter = iter(train_loader)

In [48]:
temp_batch = next(temp_batch_iter)
clip = temp_batch[0]
label = temp_batch[1]
print("Clip size: {0}".format(clip.size()))
print(label)
print(clip.max())
dataloader.play_video(clip[0])
print('Finished')

Clip size: torch.Size([4, 3, 32, 112, 112])
tensor([ 8, 23, 14, 10])
tensor(255., device='cuda:0', dtype=torch.float64)
Finished


# **2. Construct the Model**

All models are implemented on `mymodels.py`.

In [3]:
net = mymodels.Classifier7().to(device) 
num_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print('Number of parameters: %d' % (num_params))
print(net)

torch.backends.cudnn.benchmark = True

Number of parameters: 11924440
Classifier7(
  (dilation): DilationBlock(
    (convlist): ModuleList(
      (0): Sequential(
        (0): Conv3DWS(3, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), bias=False)
        (1): GroupNorm(16, 64, eps=1e-05, affine=True)
        (2): ReLU(inplace=True)
      )
      (1): Sequential(
        (0): Conv3DWS(3, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), dilation=(3, 1, 1), bias=False)
        (1): GroupNorm(16, 64, eps=1e-05, affine=True)
        (2): ReLU(inplace=True)
      )
      (2): Sequential(
        (0): Conv3DWS(3, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), dilation=(5, 1, 1), bias=False)
        (1): GroupNorm(16, 64, eps=1e-05, affine=True)
        (2): ReLU(inplace=True)
      )
    )
  )
  (norm1): GroupNorm(16, 64, eps=1e-05, affine=True)
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3DWS(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=

# **3. Train the Model**

## 3.1 Set hyperparameters, optimizer, loss, etc.


In [4]:
# Number of epoch
num_epochs = 50
# Learning rate
lr = 0.001
# Loss functions
criterion = torch.nn.CrossEntropyLoss()
# Setup optimizers
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4, nesterov=True)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

## 3.2 Train the model


In [5]:
current_epoch=1

In [6]:
# Lists to keep track of progress
train_loss_list = []
val_loss_list = []
train_accr_list = []
val_accr_list = []
label_list = range(0, 24)
zero_list = [0]*24
train_accr_dict = dict(zip(label_list, zero_list))
val_accr_dict = dict(zip(label_list, zero_list))

In [8]:
# Training Loop
print("Starting Training Loop...")
trainval_loaders = {'train': train_loader, 'val': val_loader}
trainval_sizes = {x: len(trainval_loaders[x].dataset) for x in ['train', 'val']}

for epoch in range(num_epochs):
    print("#################################### Epoch: {}/{} ####################################".format(epoch + 1, num_epochs))
    for phase in ['train', 'val']:
        start_time = time.time()

        iters = 0
        running_loss = 0.0
        running_corrects = 0.0
        total_num = 0

        if phase == 'train':
            net.train()
            #scheduler.step()
        else:
            net.eval()

        for inputs, labels in trainval_loaders[phase]:
            inputs = Variable(inputs, requires_grad=True).to(device=device, dtype=torch.float)
            labels = Variable(labels).to(device=device, dtype=torch.int64)
            b_size = inputs.size(0)
            optimizer.zero_grad()

            if phase == 'train':
                outputs = net(inputs)
            else:
                with torch.no_grad():
                    outputs = net(inputs)

            probs = nn.Softmax(dim=1)(outputs)
            preds = torch.max(probs, 1)[1]
            loss = criterion(outputs, labels)

            if phase == 'train':
                loss.backward()
                optimizer.step()

            iters += 1
                
            total_num += b_size
        
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / total_num
            epoch_acc = running_corrects.double() / total_num
            
            print("\r[{0}]  Batch: {1}/{2} / Loss: {3} / Acc: {4}"
                  .format(phase, iters, trainval_sizes[phase]//batch_size, epoch_loss, epoch_acc), end='                   ')

        if phase == 'train':
            train_loss_list.append(epoch_loss)
            train_accr_list.append(epoch_acc)
        else:
            val_loss_list.append(epoch_loss)
            val_accr_list.append(epoch_acc)

        
        stop_time = time.time()
        print()
        print("Execution time: " + str(stop_time - start_time))
    
    save(net, epoch+1)

Starting Training Loop...
#################################### Epoch: 1/50 ####################################
[train]  Batch: 636/635 / Loss: 2.1648547308115895 / Acc: 0.3314982304364923                    
Execution time: 382.99021673202515
[val]  Batch: 170/169 / Loss: 1.805237026558709 / Acc: 0.3917525773195876                     
Execution time: 43.38793921470642
Saving... Done!
#################################### Epoch: 2/50 ####################################
[train]  Batch: 636/635 / Loss: 1.0558167158783864 / Acc: 0.6362563900904443                   
Execution time: 426.8255512714386
[val]  Batch: 170/169 / Loss: 1.0909516488329476 / Acc: 0.6067746686303387                   
Execution time: 46.121655225753784
Saving... Done!
#################################### Epoch: 3/50 ####################################
[train]  Batch: 636/635 / Loss: 0.705988154560181 / Acc: 0.7463625639009044                    
Execution time: 422.48017168045044
[val]  Batch: 170/169 / Loss: 0.8

[val]  Batch: 170/169 / Loss: 0.25601682870131703 / Acc: 0.9189985272459499                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
Execution time: 35.0263454914093
Saving... Done!
#################################### Epoch: 40/50 ####################################
[train]  Batch: 636/635 / Loss: 0.03009890825599445 / Acc: 0.9897758552890287          

# **4. Save / Load the model**

## 4.1 Save the model

In [7]:
def save(net, epoch):
    global loss_list, optimizer
    print('Saving...', end=' ')
    state = {
        'epoch': current_epoch,
        'net': net.state_dict(),
        'train_loss_list' : train_loss_list,
        'val_loss_list' : val_loss_list,
        'train_accr_list' : train_accr_list,
        'val_accr_list' : val_accr_list,
        'optimizer': optimizer.state_dict(),
        }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/ckpt.pth')
    print("Done!")

In [ ]:
print("Current epoch : {}".format(current_epoch))
save(net, current_epoch)

## 4.2 Load the model

In [7]:
def load(netG):
    global current_epoch, train_loss_list, val_loss_list, train_accr_list, val_accr_list, optimizer
    print('Loading...', end=' ')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'], strict=True)
    current_epoch = checkpoint['epoch']
    train_loss_list = checkpoint['train_loss_list'],
    val_loss_list = checkpoint['val_loss_list'],
    train_accr_list = checkpoint['train_accr_list'],
    val_accr_list = checkpoint['val_accr_list'],
    optimizer.load_state_dict(checkpoint['optimizer']),
    print("Done!")

In [12]:
load(net)
train_loss_list = train_loss_list[0]
train_accr_list = train_accr_list[0]
val_loss_list = val_loss_list[0]
val_accr_list = val_accr_list[0]

Loading... Done!
